In [1]:
import xarray as xr
import distributed

In [2]:
cluster = distributed.LocalCluster(n_workers=4, threads_per_worker=1, local_directory='/tmp')
client = distributed.Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:41015 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 16.62 GB


# Split ERA5 into training / test

In [3]:
ds_era5 = xr.open_dataset('../data/raw/era5/regridded_merged.nc').chunk({'time': 2})

In [4]:
ds_era5 = ds_era5.rename({'lat': 'latitude', 'lon': 'longitude'})

In [5]:
ds_era5_train = ds_era5.sel(time=slice('2017-01-01 00:00', '2018-12-31 12:00'))[['t2m']]
ds_era5_test = ds_era5.sel(time=slice('2019-01-01 00:00', '2019-12-31 12:00'))[['t2m']]

In [7]:
ds_era5_train.to_zarr(
    '../data/processed/era5/ds_train',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)
ds_era5_test.to_zarr(
    '../data/processed/era5/ds_test',
    encoding={
        't2m': {'dtype': 'float32', 'scale_factor': 1.0, 'add_offset': 0.0},
    }
)